# Hive assignment 1

## Step 4. Complete the assignment

In [1]:
%%writefile task1_create_external_table.hql
-- Create external table posts_sample_external with suitable values
-- Your code here
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;

CREATE EXTERNAL TABLE posts_sample_external 
(Id String, 
 year String,
 month String) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' 
WITH SERDEPROPERTIES ('input.regex'='^<row.*?(?=.*\\bId=\"(\\d+)\").*(?<=\\bCreationDate\\b=\")(\\d*)-(\\d*).*') 
LOCATION '/data/stackexchange1000/posts'
TBLPROPERTIES ("skip.header.line.count"="1");   

Overwriting task1_create_external_table.hql


Make sure that you have created your table correctly. Select the first 10 posts from the dataset.

In [3]:
%%writefile task1_check_select.hql

-- Write select query for the first 10 rows
-- Your code here
use demodb;
SELECT * FROM posts_sample_external LIMIT 10;

Overwriting task1_check_select.hql


Create managed table `posts_sample`. Create the partition by the month and by the year. 

In [19]:
%%writefile task1_create_managed_table.hql
-- create managed table
-- Check that this table contains info about year and month
-- Your code here

USE demodb;
DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (Id string) 
PARTITIONED BY (year string, month string);   

Overwriting task1_create_managed_table.hql


Populate data from the table `posts_sample_external` to the table `posts_sample`. Don't forget about the partitioning rules!

In [17]:
%%writefile task1_insert_table.hql

-- Insert data to the managed table
SET hive.exec.max.dynamic.partitions=512;
SET hive.exec.max.dynamic.partitions.pernode=256;

USE demodb;

-- filling managed posts table from external one
SET hive.exec.dynamic.partition.mode=nonstrict;

-- Your code here for inserting data

INSERT OVERWRITE TABLE POSTS_SAMPLE 
PARTITION (year,month)
SELECT Id,year,month
FROM posts_sample_external
WHERE YEAR IS NOT NULL and month IS NOT NULL;


Overwriting task1_insert_table.hql


Make sure that your table contains appropriate data about posts

In [9]:
%%writefile task1_watch_new_table.hql
-- Your code here
SELECT * FROM demodb.posts_sample LIMIT 10;

Overwriting task1_watch_new_table.hql


Take the third row of the dataset in the ascending order for the posts (firstly by year, after that by month)

In [11]:
%%writefile task1_result.hql
-- Your code here
use demodb;
select concat_ws("\t",year,m,CAST(line_count AS STRING))
from
(select year, concat(year,"-",month) as m,count(*) as line_count,
 ROW_NUMBER() OVER (ORDER BY year,month) as row
from POSTS_SAMPLE
group by year,month
) as posts
where row=3;


Overwriting task1_result.hql


## Step 5. Submission part. Do not touch!! And simple run all cells below!

Copy your notebook from the steps <a href="#Step-4.-Complete-the-assignment">Step 4</a> and <a href="#Step-5.-Submission-part.-Do-not-touch!!-And-simple-run-all-cells-below!">Step 5</a> to the new notebook. Run all the cells! And submit the copied notebook!

In [20]:
!cat task1_create_external_table.hql > task1.hql
!cat task1_create_managed_table.hql >> task1.hql
!cat task1_insert_table.hql >> task1.hql
!cat task1_result.hql >> task1.hql

Take a look at your submission query!

In [21]:
!cat task1.hql

-- Create external table posts_sample_external with suitable values
-- Your code here
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;

CREATE EXTERNAL TABLE posts_sample_external 
(Id String, 
 year String,
 month String) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' 
WITH SERDEPROPERTIES ('input.regex'='^<row.*?(?=.*\\bId=\"(\\d+)\").*(?<=\\bCreationDate\\b=\")(\\d*)-(\\d*).*') 
LOCATION '/data/stackexchange1000/posts'
TBLPROPERTIES ("skip.header.line.count"="1");   -- create managed table
-- Check that this table contains info about year and month
-- Your code here

USE demodb;
DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (Id string) 
PARTITIONED BY (year string, month string);   
-- Insert data to the managed table
SET hive.exec.max.dynamic.partitions=512;
SET hive.exec.max.dynamic.partitions.pernode=256;

USE demodb

In [15]:
%%javascript

$(document).ready(function() {
    console.log('Ready');
    
    
    function is_hive_command(list_tokens) {
        return list_tokens.indexOf('hive') > -1 && 
             list_tokens.indexOf('f') > -1 &&
             list_tokens.indexOf('-') > -1 && 
             list_tokens.indexOf('!') > -1 &&
             list_tokens.indexOf('hql') > -1 && 
             list_tokens.indexOf('writefile') == -1;
    } 
    
    function collectText(input_tag) {

        var result_string = [];
        $.each($(input_tag).children(), function(index, child) {
            result_string.push($(child).text());
        });
        return [result_string, is_hive_command(result_string)];
    };
    
    var filtered_results = $(".cell.code_cell.rendered").filter(function(index, element) {
        var out = collectText($(element).find('.CodeMirror-line').find('span'));
        console.log(out);
        return collectText($(element).find('.CodeMirror-line').find('span'))[1];
    });
    $(filtered_results).remove();
});

<IPython.core.display.Javascript object>

In [22]:
%%bash
hive -f task1.hql

2008	2008-10	73



Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 1.064 seconds
OK
Time taken: 2.131 seconds
OK
Time taken: 0.642 seconds
OK
Time taken: 0.031 seconds
OK
Time taken: 3.663 seconds
OK
Time taken: 0.328 seconds
OK
Time taken: 0.027 seconds
Query ID = jovyan_20190930011616_c233664a-f10b-4c28-b082-19e76369640f
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1569802179163_0014, Tracking URL = http://d37663a6e069:8088/proxy/application_1569802179163_0014/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_156980217916

Congratulations! You have completed the assignment! Now you can submit it to the system and get your results!

Copy your notebook from the steps <a href="#Step-4.-Complete-the-assignment">Step 4</a> and <a href="#Step-5.-Submission-part.-Do-not-touch!!-And-simple-run-all-cells-below!">Step 5</a> to the new notebook. Run all the cells! And submit the copied notebook!